In [5]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.sparse import diags
from scipy.linalg import lu
from scipy import linalg
import scipy.stats as ss
import math

class FiniteDifferences(object):
    def __init__(self, S0, K, r, T, sigma, Smax, M, N):
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.sigma = sigma
        self.Smax = Smax
        self.M, self.N = int(M), int(N)
        self.dS = Smax / float(self.M)
        self.dt = T / float(self.N)
        self.i_values = np.arange(self.N+1)
        self.j_values = np.arange(self.M+1)
        self.grid = np.zeros(shape=(self.M+1, self.N+1))
        self.dtv=np.arange(0, self.T+self.dt, self.dt)
        self.dsv=np.arange(0,self.Smax+self.dS,self.dS)

    def _setup_boundary_conditions_(self):
#Here we have the lower boundary, upper boundary and the terminal condition based on indicator function 
        self.grid[0, :] = 0
        self.grid[-1, :] = self.Smax*np.exp(-self.r* self.dtv)
        self.grid[:, -1] = np.arange(0, self.Smax+self.dS, self.dS)
        for j in range(len(self.grid[5, :])):
            if self.grid[j, -1] > self.K:
                self.grid[j, -1] = self.grid[j, -1]
            else:
                self.grid[j, -1] = 0

    def _setup_coefficients_(self):
#Here we have our coefficients that are used for matrix M1 and M2
        self.alpha = .25 * self.dt * (self.sigma**2 * self.dsv**2 - self.r * self.dsv)
        self.beta = -.5 * self.dt * (self.sigma**2 * self.dsv**2 + self.r)
        self.gamma = .25 * self.dt * (self.sigma**2 * self.dsv**2 + self.r*self.dsv)

        m12=np.diag(1-self.beta, k=0)
        m11=np.diag(-self.gamma[0:-1], k=1)
        m13=np.diag(-self.alpha[1:], k=-1)

        m22=np.diag(1+self.beta, k=0)
        m21=np.diag(self.gamma[0:-1], k=1)
        m23=np.diag(self.alpha[1:], k=-1)

        self.M1=m12+m11+m13
        self.M2=m22+m21+m23

    def _traverse_grid_(self):
#We solve our matrix by inverting M1, we can follow process by seeing the "Handmade calculus" made in .pdf
        for j in range(len(self.grid[:, 5])-1, 0, -1):
            inv=np.linalg.inv(self.M1)
            aaa=np.dot(inv,self.M2)
            self.grid[:,j-1] = np.dot(aaa, self.grid[:,j])


    def _interpolate_(self):
        return np.interp(self.S0,np.arange(0, self.Smax+self.dS, self.dS),self.grid[:, 0])

    def price(self):
        self._setup_boundary_conditions_()
        self._setup_coefficients_()
        self._traverse_grid_()
        return self._interpolate_()

option = FiniteDifferences(100, 100, 0.03, 1, 0.2, 200, 100, 100)
#S = 100, K = 100, r = 3%, maturity = 1 year, volatility = 20%, Upper boundary = 200, matrix(M=100, N=100)
print(option.price())

50.29307325792955
